In [1]:
from Datasets import TokenizedChromaDataset
import numpy as np
from torch.utils.data import DataLoader, Subset
import sys
sys.path.insert(0, '..')
from transformer.models import EncoderOnlyWrapper, EncoderModel
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import torch
from tqdm import tqdm
import os

In [2]:
npz_path = '../data/augmented_and_padded_data.npz'
dataset = TokenizedChromaDataset(npz_path)

train_percentage = 0.9
split_idx = int( len(dataset)*train_percentage )

train_set = Subset(dataset, range(0,split_idx))
test_set = Subset(dataset, range(split_idx, len(dataset)))

batch_size = 8
epochs = 1000

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [3]:
src_vocab_size = 2**12
tgt_vocab_size = 2**12
d_model = 512
num_heads = 8
num_layers = 8
d_ff = 1024
max_seq_length = 129
dropout = 0.3

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoderModel = EncoderModel(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

transformer = EncoderOnlyWrapper(encoderModel)

transformer = transformer.to(dev)

In [4]:
criterion = CrossEntropyLoss(ignore_index=0)
optimizer = Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

# keep best validation loss for saving
best_val_loss = np.inf
save_dir = '../saved_models/encoderOnly_one_hot/'
encoder_path = save_dir + 'encoderOnly_one_hot.pt'
os.makedirs(save_dir, exist_ok=True)

for epoch in range(epochs):
    train_loss = 0
    running_loss = 0
    samples_num = 0
    running_accuracy = 0
    accuracy = 0
    with tqdm(train_loader, unit='batch') as tepoch:
        tepoch.set_description(f"Epoch {epoch} | trn")
        for melodies, chords in tepoch:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            optimizer.zero_grad()
            # output = transformer(melodies)
            output = transformer(chords)
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords.contiguous().view(-1))
            loss.backward()
            optimizer.step()
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            train_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
            tepoch.set_postfix(loss=train_loss, accuracy=accuracy) # tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
    # validation
    with torch.no_grad():
        val_loss = 0
        running_loss = 0
        samples_num = 0
        running_accuracy = 0
        accuracy = 0
        print('validation...')
        for melodies, chords in test_loader:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            # output = transformer(melodies)
            output = transformer(chords)
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords.contiguous().view(-1))
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            val_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
        if best_val_loss > val_loss:
            print('saving!')
            best_val_loss = val_loss
            torch.save(transformer.state_dict(), encoder_path)
        print(f'validation: accuracy={accuracy}, loss={val_loss}')

Epoch 0 | trn: 100%|██████████████████████████████████████████████████████████████████████████████████| 600/600 [02:06<00:00,  4.76batch/s, accuracy=0.253, loss=0.171]


validation...
saving!
validation: accuracy=0.2899486597728232, loss=0.02863294432746984


Epoch 1 | trn: 100%|█████████████████████████████████████████████████████████████████████████████████| 600/600 [02:14<00:00,  4.45batch/s, accuracy=0.292, loss=0.0203]


validation...
saving!
validation: accuracy=0.29458818738455733, loss=0.009267136000278519


Epoch 2 | trn: 100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [02:19<00:00,  4.31batch/s, accuracy=0.294, loss=0.00734]


validation...
saving!
validation: accuracy=0.29540264991200876, loss=0.004527520319782407


Epoch 3 | trn: 100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [02:18<00:00,  4.32batch/s, accuracy=0.295, loss=0.00352]


validation...
saving!
validation: accuracy=0.29578079322832584, loss=0.0028507410046053915


Epoch 4 | trn:  76%|████████████████████████████████████████████████████████████▍                   | 453/600 [01:45<00:34,  4.30batch/s, accuracy=0.296, loss=0.00195]


KeyboardInterrupt: 